In [29]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import lightgbm as lgb
import pandas as pd

train = pd.read_csv('train.csv')
df_train = pd.read_csv('df_train.csv', index_col='row_id')
df_test = pd.read_csv('df_test.csv', index_col='row_id')
target_train = pd.read_csv('target_train.csv', index_col='row_id').squeeze()
target_test = pd.read_csv('target_test.csv', index_col='row_id').squeeze()

In [30]:
def smape_loss(y_true, y_pred):
    """
    Custom SMAPE loss function for LightGBM
    """
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def smape_cv(model, df_train, target_train):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    for _, (train_index, test_index) in enumerate(tscv.split(df_train), start=1):
        # Create training and validation datasets
        X_train = df_train.iloc[train_index]
        y_train = target_train.iloc[train_index]
        X_valid = df_train.iloc[test_index]
        y_valid = target_train.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='mape', verbose=False)

        # Make predictions
        y_pred = np.exp(model.predict(X_valid)) * X_valid['microbusiness_density']
        y_valid = np.exp(y_valid) * X_valid['microbusiness_density']

        # Calculate SMAPE score
        smape = smape_loss(y_valid, y_pred)
        smape_list.append(smape)
    
    print(f'Mean SMAPE: {np.mean(smape_list):.4f}')
    return np.mean(smape_list)

def lgbm_objective(trial, df_train, target_train):
    """
    Objective function for LightGBM optimization without rolling window cross-validation.
    """
    params = {
        'n_estimators'      : 200,
        'verbose'           : -1,
        'objective'         : 'regression_l1',
        'random_state'      : 42,
        'extra_trees'       : True,
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'         : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'         : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'         : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'        : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf'  : trial.suggest_int('min_data_in_leaf', 5, 250),
        'early_stopping_round': 50,}

    model = lgb.LGBMRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score

In [31]:
import optuna
import pandas as pd

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: lgbm_objective(trial, df_train, target_train), n_trials=20)

# Print the best trial
print(f"Best trial: {study.best_trial.value} with parameters {study.best_trial.params}")

[I 2023-04-16 04:03:04,336] A new study created in memory with name: no-name-79375568-c738-477b-9927-23ebdb2f8d7c
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l2 is set=2.9497479075951083, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.9497479075951083
[LightGBM] [Warning] lambda_l1 is set=2.5162708518776586, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.5162708518776586
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l2 is set=2.9497479075951083, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.9497479075951083
[LightGBM] [Warning] lambda_l1 is set=2.5162708518776586, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.5162708518776586
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l2 is set=2.9497479075951083, reg_lambda=0.0 will be ignored. Current va

[I 2023-04-16 04:03:36,167] Trial 0 finished with value: 1.743668328012426 and parameters: {'colsample_bytree': 0.9101158510912557, 'colsample_bynode': 0.7950496453512623, 'max_depth': 8, 'learning_rate': 0.01822180689543358, 'lambda_l1': 2.5162708518776586, 'lambda_l2': 2.9497479075951083, 'num_leaves': 445, 'min_data_in_leaf': 14}. Best is trial 0 with value: 1.743668328012426.


Mean SMAPE: 1.7437
[LightGBM] [Warning] min_data_in_leaf is set=113, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=113
[LightGBM] [Warning] lambda_l2 is set=1.1082355556861707, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1082355556861707
[LightGBM] [Warning] lambda_l1 is set=4.833291533136985, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.833291533136985
[LightGBM] [Warning] min_data_in_leaf is set=113, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=113
[LightGBM] [Warning] lambda_l2 is set=1.1082355556861707, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1082355556861707
[LightGBM] [Warning] lambda_l1 is set=4.833291533136985, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.833291533136985
[LightGBM] [Warning] min_data_in_leaf is set=113, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=113
[LightGBM] [Warning] lambda_l2 is set=1.1082355556861707, reg_lambda=0.0 will b

[I 2023-04-16 04:03:43,156] Trial 1 finished with value: 1.7594763742191233 and parameters: {'colsample_bytree': 0.7869775799988069, 'colsample_bynode': 0.3231580682880848, 'max_depth': 3, 'learning_rate': 0.0913837832744824, 'lambda_l1': 4.833291533136985, 'lambda_l2': 1.1082355556861707, 'num_leaves': 912, 'min_data_in_leaf': 113}. Best is trial 0 with value: 1.743668328012426.


...

Mean SMAPE: 1.7464
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] lambda_l2 is set=4.3448241800941165, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.3448241800941165
[LightGBM] [Warning] lambda_l1 is set=0.6664869719924802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6664869719924802
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] lambda_l2 is set=4.3448241800941165, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.3448241800941165
[LightGBM] [Warning] lambda_l1 is set=0.6664869719924802, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6664869719924802
[LightGBM] [Warning] min_data_in_leaf is set=136, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=136
[LightGBM] [Warning] lambda_l2 is set=4.3448241800941165, reg_lambda=0.0 wi

[I 2023-04-16 04:10:35,045] Trial 18 finished with value: 1.7457025152486119 and parameters: {'colsample_bytree': 0.9630255761229617, 'colsample_bynode': 0.7318270917587809, 'max_depth': 5, 'learning_rate': 0.057252551852280856, 'lambda_l1': 0.6664869719924802, 'lambda_l2': 4.3448241800941165, 'num_leaves': 763, 'min_data_in_leaf': 136}. Best is trial 0 with value: 1.743668328012426.


Mean SMAPE: 1.7457
[LightGBM] [Warning] min_data_in_leaf is set=166, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=166
[LightGBM] [Warning] lambda_l2 is set=7.4992739745951456, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.4992739745951456
[LightGBM] [Warning] lambda_l1 is set=2.7182194656122167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.7182194656122167
[LightGBM] [Warning] min_data_in_leaf is set=166, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=166
[LightGBM] [Warning] lambda_l2 is set=7.4992739745951456, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.4992739745951456
[LightGBM] [Warning] lambda_l1 is set=2.7182194656122167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.7182194656122167
[LightGBM] [Warning] min_data_in_leaf is set=166, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=166
[LightGBM] [Warning] lambda_l2 is set=7.4992739745951456, reg_lambda=0.0 wi

[I 2023-04-16 04:11:07,270] Trial 19 finished with value: 1.743683551110469 and parameters: {'colsample_bytree': 0.9964136059729585, 'colsample_bynode': 0.5756934790502, 'max_depth': 7, 'learning_rate': 0.060227554758627434, 'lambda_l1': 2.7182194656122167, 'lambda_l2': 7.4992739745951456, 'num_leaves': 582, 'min_data_in_leaf': 166}. Best is trial 0 with value: 1.743668328012426.


Mean SMAPE: 1.7437
Best trial: 1.743668328012426 with parameters {'colsample_bytree': 0.9101158510912557, 'colsample_bynode': 0.7950496453512623, 'max_depth': 8, 'learning_rate': 0.01822180689543358, 'lambda_l1': 2.5162708518776586, 'lambda_l2': 2.9497479075951083, 'num_leaves': 445, 'min_data_in_leaf': 14}


In [32]:
params = study.best_trial.params
LGBM = lgb.LGBMRegressor(**params)

In [33]:
LGBM.fit(df_train, target_train)
y_pred = np.exp(LGBM.predict(df_test)) * df_test.microbusiness_density

[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l1 is set=2.5162708518776586, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.5162708518776586
[LightGBM] [Warning] lambda_l2 is set=2.9497479075951083, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.9497479075951083


In [34]:
pred = y_pred.values
pred

array([3.35343867, 8.52822966, 1.19579881, ..., 3.90972377, 3.13056649,
       1.80607769])

In [35]:
my_cfips = df_train['cfips'].unique()
train = train[train.cfips.isin(my_cfips)]
y_true = train[train.row_id.str.contains('2022-07-01')].microbusiness_density.values
print(smape_loss(y_true, pred))

2.250033742275787


In [37]:
target_val = train[train.row_id.str.contains('2022-07-01')][['row_id', 'microbusiness_density']]
target_val.to_csv('target_val.csv')

In [ ]:
# clip_err = True

# if clip_err:
#     df_record = y_pred.merge(y_base, on = 'row_id', how='inner')
#     df_record = df_record.merge(y_true, on = 'row_id', how='inner')
#     def smape_(y_true, y_pred):
#         return 100  * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
#     base_err = df_record.apply(lambda x: smape_(x.y_true, x.y_base), axis=1)
#     pred_err = df_record.apply(lambda x: smape_(x.y_true, x.microbusiness_density), axis=1)
#     blacklist = base_err[(base_err + 1e-3) < pred_err].index
#     y_pred.iloc[blacklist] = y_base.iloc[blacklist]
#     avg_base_err = base_err.mean()
#     avg_pred_err = pred_err.mean()
#     print(avg_base_err, avg_pred_err)
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')
# else:
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')